In [1]:
import pandas as pd
import random
from collections import defaultdict, namedtuple

In [2]:
# =========================================================================
# PARÂMETROS GERAIS
# =========================================================================
dias_semana = ["SEG", "TER", "QUA", "QUI", "SEX"]
tempos = [1,2,3,4,5]

In [3]:
# =========================================================================
# 1) LEITURA DOS ARQUIVOS
# =========================================================================
df_prof = pd.read_excel("professores.xlsx")
df_demanda_raw = pd.read_excel("demanda.xlsx")

In [4]:
# =========================================================================
# 2) CRIAR DICIONÁRIO DE DISPONIBILIDADE
#    disponib[prof][turno][dia][tempo] = True/False
# =========================================================================
disponib = {}
for i, row in df_prof.iterrows():
    servidor = row["SERVIDOR"]
    turno    = row["TURNO"]
    if servidor not in disponib:
        disponib[servidor] = {}
    if turno not in disponib[servidor]:
        disponib[servidor][turno] = {}
    for d in dias_semana:
        if d not in disponib[servidor][turno]:
            disponib[servidor][turno][d] = {}
        for t in tempos:
            col = f"{d}{t}"
            if col in row:
                val = row[col]
                disponib[servidor][turno][d][t] = (val == 1)
            else:
                disponib[servidor][turno][d][t] = False

In [5]:
# =========================================================================
# 3) FILTRAR "APOIO" E MONTAR DEMANDA
# =========================================================================
df_demand = df_demanda_raw[df_demanda_raw["DISC"] != "APOIO"].copy()
Pedido = namedtuple("Pedido", "prof turno nivel serie turma disc ch")
demanda = []
for i, row in df_demand.iterrows():
    demanda.append(Pedido(
        prof  = row["SERVIDOR"],
        turno = row["TURNO"],
        nivel = row["NIVEL"],
        serie = str(row["SERIE/ANO"]),
        turma = str(row["TURMA"]),
        disc  = row["DISC"],
        ch    = int(row["CH.TURMA"])
    ))

In [6]:
# =========================================================================
# 4) ESTRUTURA DE ALOCAÇÃO
#    horario[turno][nivel][dia][(serie,turma)] = { tempo: (prof, disc) }
# =========================================================================
def cria_horario_vazio(demanda):
    """Gera a estrutura vazia para todos os (turno, nivel, dia, serie, turma) presentes na demanda."""
    horario = {}
    combos = set()
    for ped in demanda:
        combos.add((ped.turno, ped.nivel))
    for (tnr, niv) in combos:
        if tnr not in horario:
            horario[tnr] = {}
        if niv not in horario[tnr]:
            horario[tnr][niv] = {}
        for d in dias_semana:
            horario[tnr][niv][d] = {}
    for ped in demanda:
        for d in dias_semana:
            if (ped.serie, ped.turma) not in horario[ped.turno][ped.nivel][d]:
                horario[ped.turno][ped.nivel][d][(ped.serie, ped.turma)] = {}
    return horario

In [7]:
# =========================================================================
# 5) FUNÇÕES DE VERIFICAÇÃO DE REGRAS
# =========================================================================
def professor_disponivel(prof, turno, dia, tempo):
    if prof not in disponib:
        return False
    if turno not in disponib[prof]:
        return False
    return disponib[prof][turno][dia][tempo]

def professor_ocupado_esse_tempo(horario, prof, turno, nivel, dia, tempo):
    for (st, dicT) in horario[turno][nivel][dia].items():
        if tempo in dicT:
            (p, _) = dicT[tempo]
            if p == prof:
                return True
    return False

def turma_ja_tem_aula(horario, turno, nivel, dia, serie, turma, tempo):
    return (tempo in horario[turno][nivel][dia][(serie, turma)])

def check_buraco(horario, ped, dia):
    dicT = horario[ped.turno][ped.nivel][dia][(ped.serie, ped.turma)]
    if not dicT:
        return False
    tempos_ocup = sorted(dicT.keys())
    if tempos_ocup[0] != 1:
        return True
    maxi = max(tempos_ocup)
    for x in range(1, maxi+1):
        if x not in dicT:
            return True
    return False

def professor_turma_ja_usou_dia(horario, ped, dia):
    """Evita que o mesmo professor tenha mais de 1 tempo na mesma turma (mesmo que de disciplinas diferentes)."""
    dicTurma = horario[ped.turno][ped.nivel][dia][(ped.serie, ped.turma)]
    for (tm, (p, _)) in dicTurma.items():
        if p == ped.prof:
            return True
    return False

def alocar_1tempo(horario, ped, dia, tempo):
    horario[ped.turno][ped.nivel][dia][(ped.serie, ped.turma)][tempo] = (ped.prof, ped.disc)

def desalocar_1tempo(horario, ped, dia, tempo):
    dicTurma = horario[ped.turno][ped.nivel][dia][(ped.serie, ped.turma)]
    if tempo in dicTurma:
        del dicTurma[tempo]

def pode_alocar(horario, ped, dia, tempo):
    if not professor_disponivel(ped.prof, ped.turno, dia, tempo):
        return False
    if professor_ocupado_esse_tempo(horario, ped.prof, ped.turno, ped.nivel, dia, tempo):
        return False
    # Regra: cada professor deve ter apenas 1 tempo nessa turma por dia
    if professor_turma_ja_usou_dia(horario, ped, dia):
        return False
    # Testa alocação temporária para checar buraco
    alocar_1tempo(horario, ped, dia, tempo)
    if check_buraco(horario, ped, dia):
        desalocar_1tempo(horario, ped, dia, tempo)
        return False
    return True

In [10]:
# =========================================================================
# 6) HEURÍSTICA PRINCIPAL DE ALOCAÇÃO (iterada)
# =========================================================================
NUM_ITERACOES = 10000  # Número de iterações da busca heurística

def alocar_demanda(horario, demanda):
    nao_alocados = []
    dem_local = list(demanda)
    random.shuffle(dem_local)
    for ped in dem_local:
        ch_rest = ped.ch
        dias_rand = list(dias_semana)
        random.shuffle(dias_rand)
        for d in dias_rand:
            if ch_rest <= 0:
                break
            tempos_rand = list(tempos)
            random.shuffle(tempos_rand)
            for t in tempos_rand:
                if ch_rest <= 0:
                    break
                if pode_alocar(horario, ped, d, t):
                    alocar_1tempo(horario, ped, d, t)
                    ch_rest -= 1
        if ch_rest > 0:
            nao_alocados.append((ped, ch_rest))
    return nao_alocados

def constroi_solucao(demanda):
    hor_temp = cria_horario_vazio(demanda)
    nao_aloc = alocar_demanda(hor_temp, demanda)
    return hor_temp, nao_aloc

# Executa várias iterações e escolhe a melhor solução (menor soma de não alocados)
melhor_horario = None
melhor_nao_alocados = None
menor_nao_alocados_total = float("inf")
for i in range(NUM_ITERACOES):
    hor_cand, nao_aloc_cand = constroi_solucao(demanda)
    total_nao_alocados = sum(x[1] for x in nao_aloc_cand)  # soma dos CH.TURMA não alocados
    if total_nao_alocados < menor_nao_alocados_total:
        menor_nao_alocados_total = total_nao_alocados
        melhor_horario = hor_cand
        melhor_nao_alocados = nao_aloc_cand

print(f"Melhor solução encontrada: {menor_nao_alocados_total} CH.TURMA não alocados.")


Melhor solução encontrada: 23 CH.TURMA não alocados.


In [11]:
# =========================================================================
# 7) CÁLCULO DE TOTAL DE SLOTS E PREENCHIMENTO
# =========================================================================
def conta_alocacoes(horario):
    used = 0
    for tnr in horario:
        for niv in horario[tnr]:
            for d in dias_semana:
                for (ser, tur), dic in horario[tnr][niv][d].items():
                    used += len(dic)
    return used

def total_slots(horario):
    total = 0
    for tnr in horario:
        for niv in horario[tnr]:
            # Para cada grupo, contar quantas turmas existem (assume que cada turma tem os mesmos slots em todos os dias)
            turmas = set()
            for d in dias_semana:
                turmas.update(horario[tnr][niv][d].keys())
            total += len(turmas) * len(dias_semana) * len(tempos)
    return total

total = total_slots(melhor_horario)
usados_heur = conta_alocacoes(melhor_horario)
vazios_heur = total - usados_heur
percent_filled_heur = (usados_heur / total) * 100

print("----------------------------------------------")
print("Resultado da busca heurística:")
print(f"Total de slots: {total}")
print(f"Slots usados: {usados_heur}")
print(f"Slots vazios: {vazios_heur}")
print(f"Percentual de preenchimento (heurística): {percent_filled_heur:.2f}%")
print("----------------------------------------------")


----------------------------------------------
Resultado da busca heurística:
Total de slots: 500
Slots usados: 293
Slots vazios: 207
Percentual de preenchimento (heurística): 58.60%
----------------------------------------------


In [12]:
# =========================================================================
# 8) MÉTODO DE AJUSTE FINO
#    Para os pedidos não alocados, forçamos a alocação nos slots restantes.
# =========================================================================
def ajuste_fino(horario, nao_alocados):
    for ped, ch_rest in nao_alocados:
        # Primeiro, tentar em dias que ainda não foram usados para essa turma
        for d in dias_semana:
            if ch_rest <= 0:
                break
            if not professor_turma_ja_usou_dia(horario, ped, d):
                alocs = horario[ped.turno][ped.nivel][d][(ped.serie, ped.turma)]
                proximo_slot = len(alocs) + 1
                if proximo_slot <= max(tempos):
                    if professor_disponivel(ped.prof, ped.turno, d, proximo_slot) and not professor_ocupado_esse_tempo(horario, ped.prof, ped.turno, ped.nivel, d, proximo_slot):
                        alocar_1tempo(horario, ped, d, proximo_slot)
                        ch_rest -= 1
        # Se ainda sobrar, permitir alocar mesmo em dias já usados (forçando)
        if ch_rest > 0:
            for d in dias_semana:
                if ch_rest <= 0:
                    break
                alocs = horario[ped.turno][ped.nivel][d][(ped.serie, ped.turma)]
                proximo_slot = len(alocs) + 1
                if proximo_slot <= max(tempos):
                    if professor_disponivel(ped.prof, ped.turno, d, proximo_slot) and not professor_ocupado_esse_tempo(horario, ped.prof, ped.turno, ped.nivel, d, proximo_slot):
                        alocar_1tempo(horario, ped, d, proximo_slot)
                        ch_rest -= 1
        # Se ainda sobrar, o pedido permanece com pendência
    return horario

if menor_sobra > 0:
    print("Aplicando ajuste fino para encaixar os tempos remanescentes...")
    melhor_horario = ajuste_fino(melhor_horario, melhor_nao_alocados)
    novos_nao = alocar_demanda(melhor_horario, demanda)
    nova_sobra = sum(x[1] for x in novos_nao)
else:
    nova_sobra = 0
print("----------------------------------------------")
usados_fino = conta_alocacoes(melhor_horario)
vazios_fino = total - usados_fino
percent_filled_fino = (usados_fino / total) * 100
print("Resultado após ajuste fino:")
print(f"Slots usados: {usados_fino}")
print(f"Slots vazios: {vazios_fino}")
print(f"Percentual de preenchimento (ajuste fino): {percent_filled_fino:.2f}%")
print("----------------------------------------------")


Aplicando ajuste fino para encaixar os tempos remanescentes...
----------------------------------------------
Resultado após ajuste fino:
Slots usados: 404
Slots vazios: 96
Percentual de preenchimento (ajuste fino): 80.80%
----------------------------------------------


In [14]:
# =========================================================================
# 9) EXPORTAÇÃO: GERAR PLANILHAS FINAL
#    (A) QUADRO POR TURMA (abas por (TURNO, NIVEL))
#    (B) QUADRO POR PROFESSOR (aba por professor)
#    (C) LISTA DETALHADA DOS SLOTS VAZIOS
# =========================================================================
def gera_df_por_turma(horario):
    colunas_por_tn = defaultdict(set)
    for tnr in horario:
        for niv in horario[tnr]:
            for d in dias_semana:
                for (ser, tur), dic in horario[tnr][niv][d].items():
                    colunas_por_tn[(tnr, niv)].add(f"{ser}-{tur}")
    dfs_por_aba = {}
    for tnr in horario:
        for niv in horario[tnr]:
            turmas_cols = sorted(list(colunas_por_tn[(tnr, niv)]))
            linhas = []
            for d in dias_semana:
                for tm in tempos:
                    row_dict = {
                        "TURNO": tnr,
                        "NIVEL": niv,
                        "DIA": d,
                        "TEMPO": tm
                    }
                    for col in turmas_cols:
                        row_dict[col] = ""
                    for (ser, tur), dic in horario[tnr][niv][d].items():
                        if tm in dic:
                            (p, ds) = dic[tm]
                            colname = f"{ser}-{tur}"
                            row_dict[colname] = f"{p}+{ds}"
                    linhas.append(row_dict)
            df_aba = pd.DataFrame(linhas)
            dfs_por_aba[f"{tnr}_{niv}"] = df_aba
    return dfs_por_aba

def gera_df_por_professor(horario):
    prof_data = defaultdict(lambda: defaultdict(list))
    prof_cols = defaultdict(set)
    for tnr in horario:
        for niv in horario[tnr]:
            for d in dias_semana:
                for (ser, tur), dic in horario[tnr][niv][d].items():
                    for tm, (p, ds) in dic.items():
                        col = f"{ser}-{tur}({ds})"
                        prof_cols[p].add(col)
                        prof_data[p][(d, tm)].append((tnr, niv, ser, tur, ds))
    dfs_por_prof = {}
    for p in prof_data:
        cols = sorted(list(prof_cols[p]))
        linhas = []
        for d in dias_semana:
            for tm in tempos:
                row_dict = {"PROFESSOR": p, "DIA": d, "TEMPO": tm}
                for c in cols:
                    row_dict[c] = ""
                if (d, tm) in prof_data[p]:
                    for (tnr, niv, ser, tur, ds) in prof_data[p][(d, tm)]:
                        cname = f"{ser}-{tur}({ds})"
                        row_dict[cname] = f"{tnr}-{niv}"
                linhas.append(row_dict)
        df_p = pd.DataFrame(linhas)
        dfs_por_prof[p] = df_p
    return dfs_por_prof

# Gerar DataFrames
dfs_turma = gera_df_por_turma(melhor_horario)
dfs_prof = gera_df_por_professor(melhor_horario)

# Salvar planilhas de turmas
with pd.ExcelWriter("QUADRO_HORARIOS_TURMAS.xlsx") as writer:
    for aba, df_ in dfs_turma.items():
        df_.to_excel(writer, sheet_name=aba[:31], index=False)

# Salvar planilhas de professores
with pd.ExcelWriter("QUADRO_HORARIOS_PROFESSORES.xlsx") as writer:
    for p, df_ in dfs_prof.items():
        sheetname = p[:31]
        df_.to_excel(writer, sheet_name=sheetname, index=False)

# Salvar lista detalhada de slots vazios
vazios_detalhe = []
for tnr in melhor_horario:
    for niv in melhor_horario[tnr]:
        for d in dias_semana:
            for (ser, tur), dic in melhor_horario[tnr][niv][d].items():
                for tm in tempos:
                    if tm not in dic:
                        vazios_detalhe.append({
                            "TURNO": tnr,
                            "NIVEL": niv,
                            "DIA": d,
                            "SERIE": ser,
                            "TURMA": tur,
                            "TEMPO": tm
                        })
df_vaz = pd.DataFrame(vazios_detalhe)
df_vaz.to_excel("NAO_ALOCADOS_TEMPOS_VAZIOS.xlsx", index=False)

print("Arquivos gerados:")
print(" - QUADRO_HORARIOS_TURMAS.xlsx (abas por Turno_Nivel)")
print(" - QUADRO_HORARIOS_PROFESSORES.xlsx (abas por professor)")
print(" - NAO_ALOCADOS_DETALHADO.xlsx (demanda não alocada, se houver)")
print(" - NAO_ALOCADOS_TEMPOS_VAZIOS.xlsx (lista detalhada dos slots vazios)")

----------------------------------------------
Total de turmas: 20 => 500 slots disponíveis.
Slots usados: 403
Tempos vazios (slots não preenchidos): 97
----------------------------------------------
